jsonl파일을 float32로 변환하고 parquet 파일로 변경

In [1]:
import json
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq

def process_and_save_data(jsonl_file_path, parquet_file_path):

    print(f"parquet 파일 생성")

    other_data_list = []
    embeddings_list = []

    try:
        with open(jsonl_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line)
                embedding_vector = data.pop('embedding')
                embeddings_list.append(np.array(embedding_vector, dtype=np.float32))
                other_data_list.append(data)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없음")
        return
    except (json.JSONDecodeError, KeyError) as e:
        print(f"JSON 처리 중 문제 발생 - {e} at line: {line[:50]}...")
        return

    # 데이터프레임 생성
    df = pd.DataFrame(other_data_list)
    
    # 임베딩 리스트를 한 열로 추가
    df['embedding'] = embeddings_list

    # 데이터 프레임을 parquet 파일로 저장
    print(f" {parquet_file_path} 로 저장")
    df.to_parquet(parquet_file_path, engine='pyarrow', compression='snappy')
    print(f"저장 완료")

jsonl_file = 'review_business_5up_with_embedded_vector.jsonl'
parquet_file = 'review_data_optimized.parquet'

# 데이터 처리 및 저장 함수 실행
process_and_save_data(jsonl_file, parquet_file)

# 데이터 로드 및 확인

try:
    df_loaded = pd.read_parquet(parquet_file)
    
    # 로드된 데이터 정보 확인
    print("\n 데이터 프레임 정보")
    print(df_loaded.info())

    print("\n 첫 5개 행 데이터")
    print(df_loaded.head())

except FileNotFoundError:
    print(f"오류 - 파일을 찾을 수 없음")

parquet 파일 생성
 review_data_optimized.parquet 로 저장
저장 완료

 데이터 프레임 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447796 entries, 0 to 447795
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    447796 non-null  object
 1   user_id      447796 non-null  object
 2   business_id  447796 non-null  object
 3   stars        447796 non-null  int64 
 4   embedding    447796 non-null  object
dtypes: int64(1), object(4)
memory usage: 17.1+ MB
None

 첫 5개 행 데이터
                review_id                 user_id             business_id  \
0  8JFGBuHMoiNDyfcxuWNtrA  smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ   
1  Xs8Z8lmKkosqW5mw_sVAoA  IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg   
2  JBWZmBy69VMggxj3eYn17Q  aFa96pz67TwOFu4Weq5Agg  kq5Ghhh14r-eCxlVmlyd8w   
3  cvQXRFLCyr0S7EgFb4lZqw  ZGjgfSvjQK886kiTzLwfLQ  EtKSTHV5Qx_Q7Aur9o4kQQ   
4  r2IBPY_E8AE5_GpsqlONyg  IKbjLnfBQtEyVzEu8CuOLg  VJEzpfLs_Jnzgqh5A_FVTg